In [1]:
import copy
import re
from collections import defaultdict
import pickle

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Topics

In [2]:
topics = pd.read_csv("../topic_words.csv.zip")
topics.head()

,meta,topic,weight,type,word
0,env,Экология в целом,5,0,экологичный
1,env,Экология в целом,5,0,природоохранный
2,env,Экология в целом,1,0,экология
3,env,Экология в целом,5,0,воздействие
4,env,Экология в целом,5,0,окружающая среда


In [3]:
topics["topic"].value_counts()
pass

In [4]:
re_dict = {}
re_advanced_dict = {}
sheets_dict = {}
sheets_advanced_dict = {}
base_dict = {}
advanced_dict = {}
importance_dict = {}

for idx, row in topics.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        sheets_advanced_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
        advanced_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    if row["type"] == 1:
        sheets_advanced_dict[row["topic"]][row["word"]] = row["weight"]
        advanced_dict[row["topic"]][row["word"]] = 0
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

In [6]:
for key, val in sheets_advanced_dict.items():
    re_advanced_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

# Expert

In [7]:
regex_expert = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [8]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [9]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [10]:
df = pd.read_excel("../topics/paragraph_max_values_50.xlsx", sheet_name="paragraph_max_values_50", index_col=0).rename(
    columns={"Экспертный топик Вера": "true_expert"}
)
df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text,max_topic_cos,max_score_cos,second_max_topic_cos,second_max_score_cos,true_expert,Экспертный топик МА,Комментарий
2023915.0,ПАО «Газпром»,"2015, 2014",Энергетика,ОУР,1942.0,Компании Группы Газпром вносят свой вклад в ре...,компания группа газпром вносить свой вклад реа...,Биоразнообразие,0.678372,NaN,NaN,Биоразнообразие,NaN,NaN
1515679.0,ПАО «Газпром»,2011,Энергетика,ЭО,761.0,"пересажено более 3 000 редких растений, в том ...",пересадить редкий растение число реликтовый ви...,Биоразнообразие,0.657152,NaN,NaN,Биоразнообразие,NaN,NaN
699878.0,Госкорпорация «Росатом»,2018,Энергетика,ИО,1288.0,Вклад в сохранение биоразнообразия мощности до...,активность радионуклид поступить сточный вода ...,Биоразнообразие,0.647845,NaN,NaN,Биоразнообразие,NaN,NaN
699878.0,Госкорпорация «Росатом»,2018,Энергетика,ИО,1288.0,"_x000c_Активность бета-активных радионуклидов,...",активность радионуклид поступить сточный вода ...,Биоразнообразие,0.647845,NaN,NaN,NaN,NaN,"сложный кейс, безопасность населения"
1383089.0,ПАО «Газпром»,2014,Энергетика,ЭО,1165.0,"«Белоплечий орлан», «Птицы Сахалина», «Реки Са...",белоплечий орлан птица сахалин река сахалин ки...,Биоразнообразие,0.647845,NaN,NaN,Биоразнообразие,NaN,NaN


In [11]:
vector = []
labels = []
found_words = []
true_expert_found_words = []
texts = []

for i, (text, expert_topic, original_text) in tqdm(
    enumerate(zip(df["cleaned_text"], df["true_expert"], df["original_text"])), total=df["cleaned_text"].shape[0]
):
    tmp = np.zeros(topics_vectors.shape[1])
    if pd.isna(text):
        continue

    if not re_dict.get(expert_topic, False):
        continue

    if len(re_dict[expert_topic].findall(text)) < 5:
        continue

    texts.append({"text": text, "topic": expert_topic, "original_text": original_text})

    text_words = {}
    for key, regex in re_dict.items():
        words = regex.findall(text)
        text_words[key] = words
        if key == expert_topic:
            true_expert_found_words.append((key, words))
        for word in words:
            tmp[word2idx[word]] = 1
    vector.append(tmp)
    labels.append(expert_topic)
    found_words.append(text_words)

  0%|          | 0/2038 [00:00<?, ?it/s]

100%|██████████| 2038/2038 [02:44<00:00, 12.41it/s]


In [12]:
pd.DataFrame(texts).to_csv("used_texts_expert.csv")

In [13]:
vector = np.vstack(vector)

In [14]:
w = pd.DataFrame(found_words)
w.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность
0,"[природоохранный, воздействие, природа, природ...","[реализация, регион, предотвращение, реализаци...","[деятельность, территория, вид, нефть, нефть, ...","[негативный, воздействие, экологический, зона,...","[негативный, воздействие, морской, морской, зо...","[охрана, окружающий, среда, негативный, воздей...","[международный, животное, международный, приро...","[российский, российский, среда, негативный, во...","[негативный, воздействие, качество, негативный...","[отношение, работа, работа, работа, работа]",...,"[отчётный, контроль, наблюдатель, контролировать]","[деятельность, отношение, проект, сообщество, ...",[],[национальный],"[экосистема, период, экосистема, совет, экосис...","[охрана, мероприятие, сохранение, безопасность...",[],"[проведение, проведение, ответственность, пров...","[вклад, деятельность, мероприятие, основа, дея...",[безопасность]
1,"[природа, природа, система, природа]","[мероприятие, технология, территория, проект, ...","[вид, работа, работа, технология, вид, работа,...","[экологический, экологический, солнечный]","[река, приток, качество, речной, вода, река, р...","[охрана, минимизация, использование, затрата, ...","[редкий, вид, животное, сохранение, миграция, ...","[растение, мероприятие, сохранение, ущерб, поч...","[качество, вода, использование, выращивание, и...","[работа, работа, работа, работа]",...,[],"[деятельность, деятельность, проект, общество,...",[],"[национальный, национальный]","[период, ущерб, период, человек, человек, госу...","[мероприятие, охрана, сохранение, ущерб, возни...",[],"[проведение, проведение, возможность, проведение]","[мероприятие, деятельность, деятельность, зада...",[]
2,"[влияние, контроль, влияние, природоохранный, ...","[контроль, проект, территория, проект, инициат...","[мощность, мощность, вид, территория, территор...","[зона, зона, управление, зона, управление, ком...","[сточный, вода, поверхностный, водный, зона, з...","[окружающий, среда, инициатива, инициатива, ок...","[сохранение, сохранение, редкий, исчезать, вид...","[поверхностный, объект, среда, контроль, сохра...","[вода, оценка, содержание, продукт, продукт, п...","[работа, работа, доля]",...,"[контроль, рамка, рамка, порядок, контроль, не...","[проект, проект, проект]",[],"[национальный, национальный]","[инфраструктура, государственный, фактор, неза...","[сохранение, сохранение, сохранение, облучение...",[],[],"[вклад, инициатива, дополнительный, инициатива...",[]
3,"[природа, экология, воздействие, система, возд...","[цель, рост, мероприятие, мониторинг, учёт, оц...","[энерджи, энерджи, нефть, нефть, работа, нефть...","[экологический, экологический, нефть, нефть, у...","[река, воздействие, морской, учёт, воздействие...","[электронный, экологический, экологический, эк...","[международный, сохранение, экологический, сох...","[сохранение, экологический, сохранение, сохран...","[воздействие, оценка, воздействие]","[отношение, работа, работа, работа, работа]",...,"[рамка, рамка, порядок, отчёт]","[отношение, акция]",[],[пол],"[человек, представитель, представитель, госуда...","[инвестировать, план, сохранение, план, сохран...",[],[],"[мероприятие, отчёт]",[]
4,"[защита, природа, система, влияние, влияние, п...","[территория, мониторинг, реализация, проект, п...","[работа, территория, область, система, террито...","[экологический, экологический, экологический, ...","[водный, озеро, река, очистка, озеро, озеро, о...","[защита, экологический, экологический, экологи...","[сохранение, охранять, природный, восстановлен...","[сохранение, природный, восстановление, реабил...","[оценка, оценка, состав]","[работа, персонал, работа, работа]",...,"[рамка, рамка]","[общество, проект, проект, проект, ц

In [15]:
p = pd.DataFrame(true_expert_found_words).rename(columns={0: "topic", 1: "words"})
p.head()

,topic,words
0,Биоразнообразие,"[международный, животное, международный, приро..."
1,Биоразнообразие,"[редкий, вид, животное, сохранение, миграция, ..."
2,Биоразнообразие,"[сохранение, сохранение, редкий, исчезать, вид..."
3,Биоразнообразие,"[международный, сохранение, экологический, сох..."
4,Биоразнообразие,"[сохранение, охранять, природный, восстановлен..."


In [16]:
print(f"{'Topic':<35} {'Average Word Length':<20} Count")
for topic, val in p.groupby("topic")["words"].apply(list).items():
    print(f"{topic:<35} {sum([len(e) for e in val]) / len(val):05.2f} {len(val)}")

Topic                               Average Word Length  Count
Биоразнообразие                     31.35 52
Вода                                51.70 43
Воздух                              27.96 28
Заинтересованные стороны            14.00 3
Закупки и антикоррупция             29.60 42
Климат                              28.92 48
Корпоративное управление            13.29 17
Лидерство                           33.50 2
Малый и локальный бизнес            08.83 23
Обучение и развитие                 41.85 53
Оплата труда                        27.50 56
Отношения с инвесторами             23.45 33
Отчетность и прозрачность           10.69 13
Поставщики в целом                  18.21 53
Поставщики. Работники               10.18 11
Поставщики. Экология                08.93 15
Потребители. Доступность            11.54 26
Потребители. Здоровье и благопо     13.07 28
Потребители. Качество и безопас     27.31 70
Потребители. Маркетинг, продажи     13.44 45
Потребители. Персональные данны     20.

In [17]:
def init_matrix(topics: pd.DataFrame) -> np.ndarray:
    matrix = np.zeros(topics_vectors.shape)
    # topic	weight	type	word
    for idx, row in topics.iterrows():
        matrix[label2idx[row["topic"]], word2idx[row["word"]]] = 1
    return matrix


def fit(matrix, vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if not label2idx.get(label, False):
                continue
            # print(label)
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector
    return matrix


def test(matrix, vectors, labels, debug=False):
    total = 0
    pred_labels = []
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        # print(np.argmax(vector @ matrix.T))
        total += label == idx2label[res]
        pred_labels.append(idx2label[res])
        if debug and label != idx2label[res]:
            print("expected", label, "result", idx2label[res], vector @ matrix.T)
    return pred_labels, total / len(labels)

In [18]:
matrix = init_matrix(topics)

In [19]:
for i in tqdm(range(10)):
    matrix = fit(matrix, vector, labels)
    # test(vector, labels, False)

pred_labels, acc = test(matrix, vector, labels, False)
acc

100%|██████████| 10/10 [00:01<00:00,  5.00it/s]


0.48121387283236994

In [20]:
matrix_df = pd.DataFrame(matrix, columns=topics["word"].tolist(), index=topics["topic"].unique().tolist())
matrix_df.head()

,экологичный,природоохранный,экология,воздействие,окружающая среда,природа,контроль,влияние,требование,документация,...,мероприятие,iso 14001,science based targets initiative sbti,верификация,целевые показатель,система экологического менеджмент,сэм,кибербезопасность,информационный,безопасность
Экология в целом,-110.0,-1160.0,-650.0,-2620.0,0.0,-480.0,-2760.0,-1210.0,-4230.0,-460.0,...,-4150.0,0.0,0.0,-20.0,0.0,0.0,-100.0,-20.0,-1260.0,-3160.0
Климат,-110.0,-1160.0,-650.0,-2620.0,0.0,-480.0,-2760.0,-1210.0,-4230.0,-460.0,...,-4150.0,0.0,0.0,-20.0,0.0,0.0,-100.0,-19.0,-1259.0,-3159.0
Энергия,-110.0,-1160.0,-650.0,-2620.0,0.0,-480.0,-2760.0,-1210.0,-4230.0,-460.0,...,-4150.0,0.0,0.0,-20.0,0.0,0.0,-100.0,-20.0,-1260.0,-3160.0
Воздух,-110.0,-659.0,-590.0,-2199.0,0.0,21.0,-2660.0,-750.0,-4130.0,-440.0,...,-3670.0,0.0,0.0,-20.0,0.0,0.0,-100.0,-20.0,-1240.0,-3000.0
Вода,-110.0,-1020.0,-610.0,-2039.0,0.0,-480.0,-2460.0,-990.0,-4089.0,-460.0,...,-3990.0,0.0,0.0,-20.0,0.0,0.0,-80.0,-20.0,-1260.0,-3040.0


In [21]:
np.save("expert.npy", matrix)
matrix_df.to_csv("expert_matrix.csv")
matrix_df.T.to_csv("expert_matrix.T.csv")

In [22]:
results = []
for col in tqdm(matrix_df.columns):
    max_val = matrix_df[col].max()[0] if type(matrix_df[col].max()) is not float else matrix_df[col].max()
    min_val = matrix_df[col].min()[0] if type(matrix_df[col].min()) is not float else matrix_df[col].min()
    if max_val > 0:
        results.append((col, max_val))
    if min_val < -2000:
        results.append((col, min_val))

results_df = pd.DataFrame(results, columns=["word", "val"])
results_df.to_csv("matrix_words.csv")

100%|██████████| 3775/3775 [00:09<00:00, 391.75it/s]


In [23]:
# calculate f1, prescion, recall
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print(
    "accuracy",
    accuracy_score(
        labels,
        pred_labels,
    ),
)
print(
    "f1",
    f1_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "precision",
    precision_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "recall",
    recall_score(
        labels,
        pred_labels,
        average="macro",
    ),
)

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy 0.48121387283236994
f1 0.2893260251698214
precision 0.4586301712066188
recall 0.3064465374700004


In [24]:
# fit(topics_vectors, list(sorted(topics["topic"].unique())))
# test(topics_vectors, list(sorted(topics["topic"].unique())))

In [25]:
topics_vectors.shape

(45, 3775)

In [26]:
df = pd.read_csv("paragraphs_banks.csv.zip")  # , usecols=["cleaned_text"], nrows=ROWS
df = df.dropna().reset_index(drop=True)
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

(127738, 7)

In [27]:
text = df.loc[100, "cleaned_text"]
# text = "Доля сотрудников, охваченных коллективными договорами".lower()


def test_algos(text: str, verbouse: bool = False) -> dict[str, str | float | list[str] | None]:
    arr = []
    result = {"text": text}
    vector = np.zeros(matrix.shape[1])

    for key, regex in re_dict.items():
        words = regex.findall(text)
        paragraph_dict = copy.deepcopy(base_dict)
        importance = 0
        paragraph_len = len(text.split())
        base_words = 0
        for word in words:
            paragraph_dict[key][word] = sheets_dict[key][word]
            importance += sheets_dict[key][word] > 1
            vector[word2idx[word]] = 1
            base_words += 1
        # if base_words / paragraph_len < 0.1 or importance < 1:
        #     arr.append((key, 0, 0, []))
        #     continue

        dist = cosine_similarity(X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())])[0][0]
        arr.append((copy.deepcopy(key), copy.deepcopy(dist), copy.deepcopy(importance), copy.deepcopy(words)))
        result = {**result, **{f"{key}_dist": dist, f"{key}_importance": importance, f"{key}_words": words}}
    if verbouse:
        print("\nexpert")
    res = vector @ matrix.T
    for idx in np.argsort(res)[::-1]:
        if verbouse:
            print(idx2label[idx], res[idx])
        result[f"{idx2label[idx]}_expert"] = res[idx]

    if verbouse:
        print("\ncos")
        for key, dist, importance, words in sorted(arr, key=lambda x: x[1], reverse=True):
            print(f"{key} расстояние {dist:.4f} важные слова {importance} {copy.deepcopy(set(words))}")
    return result

In [28]:
text = df.loc[100, "cleaned_text"]
print(text)
test_algos(text, True)
a = 0

привлечение средство население

expert
Социальные инвестиции и благотв -3798.0
Сотрудники. Безопасность и охра -4520.0
Потребители. Качество и безопас -4596.0
Закупки и антикоррупция -4640.0
Биоразнообразие -4680.0
Потребители. Ценовая политика -4680.0
Потребители. Доступность -4716.0
Сотрудники. Профсоюз и Коллекти -4720.0
Экология в целом -4760.0
Потребители. Здоровье и благопо -4840.0
Сотрудники. Волонтерство -4880.0
Сотрудники. Здоровье и благопол -4920.0
Поставщики в целом -4920.0
Отношения с инвесторами -4920.0
Обучение и развитие -5000.0
Потребители. Персональные данны -5000.0
Потребители. Удовлетворенность -5040.0
Потребители. Сервис и коммуника -5040.0
Заинтересованные стороны -5040.0
Устойчивое развитие -5080.0
Энергия -5080.0
Воздух -5080.0
Отчетность и прозрачность -5080.0
Права человека -5080.0
Климат -5120.0
Малый и локальный бизнес -5120.0
Корпоративное управление -5120.0
Потребители. Маркетинг, продажи -5156.0
Сотрудники. Вовлеченность и мот -5160.0
Риски -5160.0
Рекуль

In [29]:
def get_scores(text: str) -> dict[str, float | None]:
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    base_words = {}
    advanced_words = {}
    vector_exp = np.zeros(matrix.shape[1])
    if type(text) is float:
        print(text, type(text))
        return {}
    paragraph_len = len(text.split())

    total_found = 0
    for key, regex in re_dict.items():
        words = regex.findall(text)
        # advanced_words_found = re_advanced_dict[key].findall(text)
        for word in words:
            found_words = True
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            paragraph_importance[key] += weight > 1

            vector_exp[word2idx[word]] = 1

        base_words[key] = len(words)
        total_found += len(words)
        # advanced_words[key] = len(advanced_words_found)

    debug_dict = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        debug_dict["total_found_words"] = total_found
        return {**res, **{"max_score_cos": None, "max_topic_cos": None, "count_words": None}, **debug_dict}

    topic_found = False
    res = {}
    for key, vector in sheets_dict.items():
        debug_dict[f"{key}_percent_of_words"] = base_words[key] / paragraph_len
        debug_dict[f"{key}_importance"] = paragraph_importance[key]
        debug_dict[f"{key}_base_words_count"] = base_words[key]
        if base_words[key] / paragraph_len < 0.1 or paragraph_importance[key] < 1:
            res[key] = 0
            continue
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
        topic_found = True

    if not topic_found:
        res = {key: 0 for key in sheets_dict}
        return {**res, **{"max_score_cos": -100, "max_topic_cos": "безтемы", "count_words": None}, **debug_dict}

    paragraph_matrix = vector_exp @ matrix.T
    best_indicies = paragraph_matrix.argsort()[-2:][::-1]
    best_topics = [idx2label[i] for i in best_indicies]
    best_scores = paragraph_matrix[best_indicies]

    res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
    max_value = res_sorted[0]
    second_values = res_sorted[1]
    if second_values[1] == 0:
        second_values = (None, None)

    return {
        **res,
        **{
            "max_topic_cos": max_value[0],
            "max_score_cos": max_value[1],
            "second_max_topic_cos": second_values[0],
            "second_max_score_cos": second_values[1],
            "max_topic_exp": best_topics[0],
            "max_score_exp": best_scores[0],
            "second_max_topic_exp": best_topics[1],
            "second_max_score_exp": best_scores[1],
            "total_found_words": total_found,
        },
        **debug_dict,
    }

In [30]:
topics = []

i = 0
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row["cleaned_text"]
    res = get_scores(text)
    res["idx"] = idx
    if res.get("max_score_cos") is None:
        # topics.append(res)
        continue
    res["cleaned_text"] = text
    res["original_text"] = row["original_text"]
    # print(res["max_score_cos"], res["max_topic_cos"])
    topics.append(res)

100%|██████████| 127738/127738 [58:15<00:00, 36.54it/s]  


In [31]:
with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [ ]:
with open("topics.pkl", "rb") as f:
    topics = pickle.load(f)

In [32]:
topics = pd.DataFrame(topics).set_index(keys="idx")

In [33]:
topics.to_csv("all_topics_banks.csv")

In [34]:
topics.to_csv("all_topics_banks.csv.zip")

In [35]:
df.head()

,Unnamed: 0,index,company,year,paragraph,original_text,cleaned_text
0,0,1481,ПАО Сбербанк,2020,0,годовой отчет 2020,годовой отчёт 2020
1,1,1481,ПАО Сбербанк,2020,1,содержание,содержание
2,2,1481,ПАО Сбербанк,2020,2,содержание 3,содержание 3
3,3,1481,ПАО Сбербанк,2020,3,обращение председателя правления оао«бпс-сберб...,обращение председатель правление
4,4,1481,ПАО Сбербанк,2020,4,27,27


In [36]:
topics.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_text,original_text,count_words
idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0.333333,0,1,0.0,0,0,годовой отчёт 2020,годовой отчет 2020,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0.000000,0,0,0.0,0,0,содержание,содержание,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0.000000,0,0,0.0,0,0,содержание 3,содержание 3,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0.000000,0,0,0.0,0,0,обращение председатель правление,обращение председателя правления оао«бпс-сберб...,NaN
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0.000000,0,0,0.0,0,0,развитие ммб,развитие ммб,NaN


In [37]:
df = df.join(topics, rsuffix="topics")

In [40]:
df.tail()

,Unnamed: 0,index,company,year,paragraph,original_text,cleaned_text,Экология в целом,Климат,Энергия,...,Корпоративное управление_base_words_count,Устойчивое развитие_percent_of_words,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,!Кибербезопасность_percent_of_words,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_texttopics,original_texttopics,count_words
127733,135849,600,Банк ИПБ (АО),2020,2981,90,90,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127734,135850,600,Банк ИПБ (АО),2020,2982,3961,3961,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127735,135851,600,Банк ИПБ (АО),2020,2983,энергоресурсы,энергоресурсы,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,энергоресурсы,энергоресурсы,NaN
127736,135852,600,Банк ИПБ (АО),2020,2984,"топливо, т (бензин, дизельное)",топливо так бензин дизельный,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,топливо так бензин дизельный,"топливо, т (бензин, дизельное)",NaN
127737,135853,600,Банк ИПБ (АО),2020,2985,110,110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.to_csv("paragrapghs_with_topics_banks.csv")
df.to_csv("paragrapghs_with_topisc_banks.csv.zip")

In [ ]:
df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)
df.head()

,Unnamed: 0,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text,...,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_texttopics,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words
0,0.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,0.0,та новая выс,тот новый выс,...,0.0,0.0,тот новый выс,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,1.0,взаимодействия,взаимодействие,...,0.0,0.0,взаимодействие,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,2.0,годовой отчет за 2020 год,годовой отчёт 2020 год,...,0.0,0.0,годовой отчёт 2020 год,NaN,NaN,Отношения с инвесторами,-4170.0,Отчетность и прозрачность,-4784.0,6.0
3,3.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,3.0,содержание,содержание,...,0.0,0.0,содержание,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,4.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(267160, 201)

In [ ]:
df = df[df["max_topic_exp"].isna()]
df = df[df["original_text"].str.len() > 100]
df.shape

(14505, 201)

In [ ]:
df.head()

,Unnamed: 0,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text,...,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words,cleaned_texttopics
46,46.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,46.0,в настоящем годовом отчете под терминами «мега...,настоящий годовой отчёт термин компания различ...,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,47.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,47.0,награды и признание годовые отчеты мегафона на...,награда признание годовой отчёт мегафон протяж...,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,60.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,60.0,в 2020 году мегафон завершил свой трехлетний с...,2020 год завершить свой трёхлетний стратегичес...,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,61.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,61.0,расскажите о тенденциях 2020 года на российско...,рассказать тенденция 2020 год российский телек...,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,70.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,70.0,"важным аспектом, влияющим на успех деятельност...",важный аспект влиять успех деятельность компан...,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col = ["company", "year", "original_text", "cleaned_text"]
col.extend(df.columns[df.columns.str.contains("_base_words_count")].tolist())

In [ ]:
df[col].sample(1_000).to_csv("paragrapghs_with_topisc_sample.csv")

In [ ]:
df.groupby("max_topic_cos").apply(lambda x: x.nlargest(30, "max_score_cos"))[
    [
        "company",
        "year",
        "sector",
        "report_type",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_topic_cos",
        "max_score_cos",
        "second_max_topic_cos",
        "second_max_score_cos",
        "max_topic_exp",
        "max_score_exp",
        "second_max_topic_exp",
        "second_max_score_exp",
    ]
].to_csv(
    "paragraph_max_values_30.csv"
)  # .reset_index()

In [ ]:
df.groupby("max_topic").first()  # ["max_score"].first()

,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text,Климат,Энергия,...,Отчетность и прозрачность,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,max_score,second_max_score,second_max_topic
max_topic,,,,,,,,,,,,,,,,,,,,,
0.140988,2244210,АО «ТВЭЛ»,2015,Энергетика,ИО,263.0,Деятельность оказание сопутствующих услуг Пост...,деятельность оказание сопутствовать услуга пос...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.143674,1350987,ПАО «Юнипро»,2018,Энергетика,ИО,4877.0,Предмет Дополнительного соглашения: 1. Стороны...,предмет дополнительный соглашение сторона дого...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.146310,457049,ПАО «Полюс»,2017,Металлургическая и горнодобывающая,ИО,5978.0,Выбытие линий электропередачи По состоянию на ...,выбытие линия электропередача по состояние июн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,3245071,ПАО «ФСК ЕЭС» («Россети»),2009,Энергетика,ОУР,162.0,3.2. Подход и результативность работ по технич...,подход результативность работа технический обс...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,1350990,ПАО «Юнипро»,2018,Энергетика,ИО,4880.0,"неполном объеме, Заказчик имеет право отказать...",неполный объём заказчик иметь право отказать п...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.845154,3765,ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,2070.0,3.4. Обучение и развитие сотрудников разработк...,обучение развитие сотрудник разработка отдельн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Удовлетворенность потребителей,0.493352
0.847566,1323153,АО «СУЭК»,"2019, 2018",Металлургическая и горнодобывающая,ОУР,1621.0,Важнейшим средством продвижения корпоративной ...,важный средство продвижение корпоративный куль...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Здоровье и благополучие сотрудн,0.571662
0.849970,2947503,Группа ЛСР,2020,Строительство,ОУР,94.0,Работа по усилению практик устойчивого развити...,работа усиление практика устойчивый развитие п...,0.0,0.0,...,0.391381,0,0.0,0.345949,0.0,0.0,0.0,Вовлеченность и мотивация,Обучение и развитие,0.575212


In [ ]:
df.to_csv("paragrapghs_with_topisc.csv")

In [ ]:
grouped = pd.DataFrame(df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(" ".join))
grouped.head()

cleaned_text
company     year max_topic                                                                     
Alcoa, Inc. 2004 Антикоррупция                москва белый калитва восток московский офис са...
                 Безопасность и охрана труда          доход европа процент общий доход компания
                 Безопасность продукта        я очень приятно представить ваш внимание первы...
                 Благотворительность          лос летучий органический соединение в тысяча м...
                 Налоги                       возрастание выброс происходить связь применени...

In [ ]:
from collections import defaultdict

topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics_df.iterrows(), total=topics_df.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

100%|██████████| 4414/4414 [00:00<00:00, 19458.61it/s]


In [ ]:
len_topics

defaultdict(dict,
            {'Экология': {1: 51, 2: 8},
             'Климат': {1: 95, 2: 24},
             'Энергия': {1: 52, 2: 11},
             'Воздух': {1: 42},
             'Вода': {1: 61, 2: 8},
             'Отходы': {1: 41, 2: 10},
             'Биоразнообразие': {1: 47},
             'Социально-культурное воздействи': {1: 3},
             'Антикоррупция': {1: 50},
             'Дивиденды и акционеры': {1: 306},
             'Инновации': {1: 104},
             'Лидерство': {1: 371},
             'Отчетность и прозрачность': {1: 530},
             'Предотвращение коррупции': {1: 92},
             'Управление рисками': {1: 214},
             'Эффективность и производительность': {1: 319},
             'Безопасность и охрана труда': {1: 349},
             'Безопасность продукта': {1: 50},
             'Благотворительность': {1: 259},
             'Инвестиции и капитальные вложения': {1: 370},
             'Налоги': {1: 81},
             'Обучение и развитие': {1: 50},
        

In [ ]:
from __future__ import annotations


def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent = count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [ ]:
scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|██████████| 8788/8788 [01:05<00:00, 135.19it/s]


In [ ]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       max_topic                                                                              
Alcoa, Inc. 2004       Антикоррупция                      москва белый калитва восток московский офис са...   
                       Безопасность и охрана труда                доход европа процент общий доход компания   
                       Безопасность продукта              я очень приятно представить ваш внимание первы...   
                       Благотворительность                лос летучий органический соединение в тысяча м...   
                       Налоги                             возрастание выброс происходить связь применени...   
                       Отношения с потребителями                 белый калитва любучан восток москва самара   
                       Отношения с работниками            вступление выброс загрязнять вещество атмосфер...   
                       Отходы                             показатель здоровье работник безопасность труд...   
                       Предотвращение коррупции           наш преимущество заключаться производить очень...   
                       Профсоюзы и коллективные договоры  показатель экологический безопасность для комп...   
                       Управление рисками                 отчёт устойчивый развитие год отчёт ход выполн...   
BIOCAD      2019, 2018 Антикоррупция                      соответствие отчёт руководство прямой речь пре...   
                       Безопасность и охрана труда        экопроект павел яко вл быть в ранний разработк...   
                       Безопасность продукта              содержание устойчивый развитие стратегия приор...   
                       Благотворительность                август россия мир уровень вовлечённость сотруд...   
                       Вода                               характер собственность форма существенный изме...   
                       Дивиденды и акционеры              сокращать отход производство упаковка снижать ...   
                       Инвестиции и капитальные вложения  самый продавать препарат одержать победа суд п...   
                       Инновации                          генферон® ацеллбия® альгерон® эфлейра® млн руб...   
                       Климат                             о компания отход удаться ускорить синтезирован...   

                                                          score  
company     year       max_topic                                 
Alcoa, Inc. 2004       Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Налоги                                 5  
                       Отношения с потребителями              5  
                       Отношения с работниками                5  
                       Отходы                                 7  
                       Предотвращение коррупции               5  
                       Профсоюзы и коллективные договоры      5  
                       Управление рисками                     5  
BIOCAD      2019, 2018 Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Вода                                   7  
                       Дивиденды и акционеры                  5  
                       Инвестиции и капитальные вложения      5  
                       Инновации                              5  
                       Климат                                 7

In [ ]:
grouped.to_csv("grouped_paragraphs.csv.zip")